In [1]:
%run functions.ipynb

Ready: 2023-05-26 20:52:20


# Формулы

In [3]:
%%time
city = 'Москва'
df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
df_all = df_all[df_all['city'] == city]
df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_requests'].transform(np.sum)
df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric'] + '. ' + 'Запросы'
df_requests = df_all.copy()

df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
df_all = df_all[df_all['city'] == city]
df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_users'].transform(np.sum)
df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric'] + '. ' + 'Пользователи'
df_users = df_all.copy()

df_100 = pd.concat([df_requests, df_users])
exclude = [5, 8, 10, 12, 13, 14, 17, 18, 19, 20, 21, 22, 25, 26, 28, 32, 38, 40, 44, 46, 51,54,56 ]
df_100 = df_100.iloc[[i for i in df_100.index.values if i not in exclude]].reset_index().iloc[:30]


## Переменные формулы
df1 = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
df1 = df1[df1['city'] == city]
df1['city_rubric_param'] = df1['city'] + '. ' + df1['rubric'] + '. ' + 'Пользователи'
df1 = df1[['date_r', 'cnt_users', 'city_rubric_param']].rename({'cnt_users':'cnt'}, axis=1)

df2 = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
df2 = df2[df2['city'] == city]
df2['city_rubric_param'] = df2['city'] + '. ' + df2['rubric'] + '. ' + 'Запросы'
df2 = df2[['date_r', 'cnt_requests', 'city_rubric_param']].rename({'cnt_requests':'cnt'}, axis=1)
df_dates_users = pd.concat([df1, df2])
df_dates_users['cnt_all'] = df_dates_users.groupby('city_rubric_param')['cnt'].transform(np.sum)
df_dates_users['city_rubric_param_cnt'] = df_dates_users['cnt_all'].astype(str) + '_' + df_dates_users['city_rubric_param']

dfx = pd.pivot_table(df_dates_users, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric_param_cnt'])
t = dfx.corr().fillna(0).astype(np.float16)

t = t.rename_axis([None]).rename_axis([None], axis=1)
t1 = t.stack()
t1 = t1.reset_index()
t1.columns = ['r1', 'r2', 'corr']
t1 = t1[t1.r1 != t1.r2]

t1_100 = df_100.merge(t1, left_on='cnt_city_rubric', right_on='r1')

t1_100['r1_cnt'] = t1_100.r1.apply(lambda x: x.split('_')[0])
t1_100['r2_cnt'] = t1_100.r2.apply(lambda x: x.split('_')[0])

t1_100.r1 = t1_100.r1.apply(lambda x: x.split('_')[1])
t1_100.r2 = t1_100.r2.apply(lambda x: x.split('_')[1])
t1_100['r1_city'] = t1_100.r1.apply(lambda x: x.split('. ')[0])
t1_100['r2_city'] = t1_100.r2.apply(lambda x: x.split('. ')[0])
t1_100['r1_rubric'] = t1_100.r1.apply(lambda x: x.split('. ')[1])
t1_100['r2_rubric'] = t1_100.r2.apply(lambda x: x.split('. ')[1])
t1_100['r1_param'] = t1_100.r1.apply(lambda x: x.split('. ')[2])
t1_100['r2_param'] = t1_100.r2.apply(lambda x: x.split('. ')[2])

t1_100.r1_cnt = t1_100.r1_cnt.astype(int)
t1_100.r2_cnt = t1_100.r2_cnt.astype(int)

#t1_100 = t1_100[t1_100.r2_cnt > 1000]

t1_100 = t1_100[['r1_city', 'r1_rubric', 'r1_param', 'r2_city', 'r2_rubric', 'r2_param', 'corr', 'r1_cnt', 'r2_cnt']]



### f1

result_city = 'Москва'
variables_city = 'Москва'
result_param = 'Запросы'
variables_param = 'Запросы'

t1_100_city = t1_100[(t1_100.r1_city == result_city) & (t1_100.r2_city == variables_city) & (t1_100.r1_param == result_param) & (t1_100.r2_param == variables_param)]
df_100['param'] = df_100.cnt_city_rubric.apply(lambda x: x.split('. ')[2])
df_100_city = df_100[(df_100.city == result_city) & (df_100.param == result_param)]

r_list = [df_100_city.rubric.iloc[:30]]


df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[:3]):
    f1_i = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f1 = pd.concat(df_list)
f1['group'] = 'f1'

### f2
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[3:6]):
    f1_i = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric)].sort_values(['corr', 'r2_cnt'], ascending=[True,False])
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f2 = pd.concat(df_list)
f2['group'] = 'f2'

### f3
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[6:9]):
    t1_c1 = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric)]
    t1_c1['corr_abs'] = t1_c1['corr'].abs()
    t1_c1.sort_values(['corr_abs', 'r2_cnt'], ascending=[False,False], inplace=True)
    f1_i = t1_c1.iloc[:]
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20].drop('corr_abs', axis=1)
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f3 = pd.concat(df_list)
f3['group'] = 'f3'

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: total: 7.78 s
Wall time: 10.2 s


In [6]:
### f7

#### органическая – для построения тренда используются смысловые ряды переменных: запросы базируются на запросах, пользователи на пользователях, динамика на динамике, схожесть на схожести


result_city = 'Москва'
variables_city = 'Москва'
result_param = 'Запросы'
variables_param = 'Запросы'


t1_100_city = t1_100[(t1_100.r1_city == result_city) & (t1_100.r2_city == variables_city)]
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[12:15]):
    f1_i = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric) & (t1_100_city.r1_param == t1_100_city.r2_param)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f7 = pd.concat(df_list).sort_values(['r1_city','r1_rubric','r1_param'])
f7['group'] = 'f7'

### f8

#### синтетическая – в формулу замешиваются показатели наибольшей предсказательной силы, без учёта их физической природы
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[15:18]):
    t1_c1 = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i = t1_c1.iloc[:]
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f8 = pd.concat(df_list).sort_values(['r1_city','r1_rubric','r1_param'])
f8['group'] = 'f8'

### f9

#### адаптационная – задействуется оптимизационный алгоритм вариативности, чередующий переменные, не допуская преобладания единственного типа.
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[18:21]):
    f1_1 = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric) & (t1_100_city.r2_param == 'Пользователи')].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_1 = f1_1[f1_1.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_1 = f1_1.iloc[:10]
    r_list.append(f1_1.r1_rubric.drop_duplicates())
    r_list.append(f1_1.r2_rubric.drop_duplicates())
    
    f1_2 = t1_100_city[(t1_100_city.r1_city == city) & (t1_100_city.r1_rubric == rubric) & (t1_100_city.r2_param == 'Запросы')].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_2 = f1_2[f1_2.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_2 = f1_2.iloc[:10]
    
    f1_i = pd.concat([f1_1, f1_2])
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f9 = pd.concat(df_list)
f9['group'] = 'f9'

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
def calc():
    city = 'Москва'
    df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
    df_all = df_all[df_all['city'] == city]
    df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
    df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
    df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_requests'].transform(np.sum)
    df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
    df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
    df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric'] + '. ' + 'Запросы'
    df_requests = df_all.copy()

    df_all = pd.read_parquet(f'files/ten_city/df_dates_users_city_agg2')
    df_all = df_all[df_all['city'] == city]
    df_all['city_rubric'] = df_all['city'] + '. ' + df_all['rubric']
    df_all = df_all[~df_all.rubric.apply(lambda x: (x is None or x.isnumeric()))]
    df_all['cnt_all'] = df_all.groupby('city_rubric')['cnt_users'].transform(np.sum)
    df_all = df_all.drop(['date_r', 'cnt_requests', 'cnt_users'], axis=1).drop_duplicates()
    df_all = df_all.sort_values(['cnt_all'], ascending=False).reset_index().reset_index().set_index('level_0').drop('index', axis=1)
    df_all['cnt_city_rubric'] = df_all['cnt_all'].astype(str) + '_' + df_all['city_rubric'] + '. ' + 'Пользователи'

    df_100 = pd.concat([df_requests, df_users])
    exclude = [5, 8, 10, 12, 13, 14, 17, 18, 19, 20, 21, 22, 25, 26, 28, 32, 38, 40, 44, 46, 51,54,56 ]
    df_100 = df_100.iloc[[i for i in df_100.index.values if i not in exclude]].reset_index()

    result_city = 'Москва'
    variables_city = 'Москва'
    result_param = 'Запросы'
    variables_param = 'Запросы'

    df_100['param'] = df_100.cnt_city_rubric.apply(lambda x: x.split('. ')[2])
    df_100_city = df_100[(df_100.city == result_city) & (df_100.param == result_param)]
    
    df_30 = df_100_city.iloc[:30]
    return df_30

df_30 = calc()

with open('files/ten_city/t1_new.pkl', 'rb') as fp:
    t1 = pickle.load(fp)

t1_100_all = df_30.merge(t1, left_on='cnt_city_rubric', right_on='r1')
del t1

t1_100_all['r1_cnt'] = t1_100_all.r1.apply(lambda x: x.split('_')[0])
t1_100_all['r2_cnt'] = t1_100_all.r2.apply(lambda x: x.split('_')[0])

t1_100_all.r1 = t1_100_all.r1.apply(lambda x: x.split('_')[1])
t1_100_all.r2 = t1_100_all.r2.apply(lambda x: x.split('_')[1])
t1_100_all['r1_city'] = t1_100_all.r1.apply(lambda x: x.split('. ')[0])
t1_100_all['r2_city'] = t1_100_all.r2.apply(lambda x: x.split('. ')[0])
t1_100_all['r1_rubric'] = t1_100_all.r1.apply(lambda x: x.split('. ')[1])
t1_100_all['r2_rubric'] = t1_100_all.r2.apply(lambda x: x.split('. ')[1])
t1_100_all['r1_param'] = t1_100_all.r1.apply(lambda x: x.split('. ')[2])
t1_100_all['r2_param'] = t1_100_all.r2.apply(lambda x: x.split('. ')[2])

t1_100_all.r1_cnt = t1_100_all.r1_cnt.astype(int)
t1_100_all.r2_cnt = t1_100_all.r2_cnt.astype(int)

t1_100_all = t1_100_all[t1_100_all.r2_cnt > 1000]

t1_100_all = t1_100_all[['r1_city', 'r1_rubric', 'r1_param', 'r2_city', 'r2_rubric', 'r2_param', 'corr', 'r1_cnt', 'r2_cnt']]

df_100_city = df_30

#### f4
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[21:24]):
    f1_i = t1_100_all[(t1_100_all.r1_rubric == rubric)].sort_values(['corr', 'r2_cnt'], ascending=[False,False])
    f1_i.sort_values(['corr', 'r2_cnt'], ascending=[False,False], inplace=True)
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f4 = pd.concat(df_list)
f4['group'] = 'f4'

### f5

df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[24:27]):
    f1_i = t1_100_all[(t1_100_all.r1_city == city) & (t1_100_all.r1_rubric == rubric) & (t1_100_all.r2_city != city) & (t1_100_all.r2_rubric != rubric)]
    f1_i.sort_values(['corr', 'r2_cnt'], ascending=[False,False], inplace=True)
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f5 = pd.concat(df_list)
f5['group'] = 'f5'


### f6
df_list = []
for rubric, city  in tqdm(list(zip(df_100_city.rubric, df_100_city.city))[9:12]):
    f1_i = t1_100_all[(t1_100_all.r1_city == city) & (t1_100_all.r1_rubric == rubric)]
    f1_i.sort_values(['corr', 'r2_cnt'], ascending=[False,False], inplace=True)
    f1_i = f1_i[f1_i.r2_rubric.apply(lambda x: (x not in pd.concat(r_list).values) & (len(x) <= 50) & (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
    f1_i = f1_i.iloc[:20]
    f1_i['avg_corr'] = f1_i['corr'].mean()
    r_list.append(f1_i.r1_rubric.drop_duplicates())
    r_list.append(f1_i.r2_rubric.drop_duplicates())
    df_list.append(f1_i)
f6 = pd.concat(df_list)
f6['group'] = 'f6'

f_all = pd.concat([f1,f2,f3,f4,f5,f6,f7,f8,f9])
f_all['r1_rubric_city'] = f_all['r1_city'] + '. ' + f_all['r1_rubric']
f_all['r2_rubric_city'] = f_all['r2_city'] + '. ' + f_all['r2_rubric']

display(f_all.groupby('group').count())
f_all = f_all[f_all.r2_rubric.apply(lambda x: (x not in f1_i.r1_rubric.values))].drop_duplicates(subset='r2_rubric', keep="first")
f_all.groupby('group').count()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,r1_city,r1_rubric,r1_param,r2_city,r2_rubric,r2_param,corr,r1_cnt,r2_cnt,avg_corr,r1_rubric_city,r2_rubric_city
group,,,,,,,,,,,,
f1,60,60,60,60,60,60,60,60,60,60,60,60
f2,60,60,60,60,60,60,60,60,60,60,60,60
f3,60,60,60,60,60,60,60,60,60,60,60,60
f4,60,60,60,60,60,60,60,60,60,60,60,60
f5,60,60,60,60,60,60,60,60,60,60,60,60
f6,60,60,60,60,60,60,60,60,60,60,60,60
f7,60,60,60,60,60,60,60,60,60,60,60,60
f8,60,60,60,60,60,60,60,60,60,60,60,60
f9,60,60,60,60,60,60,60,60,60,60,60,60


,r1_city,r1_rubric,r1_param,r2_city,r2_rubric,r2_param,corr,r1_cnt,r2_cnt,avg_corr,r1_rubric_city,r2_rubric_city
group,,,,,,,,,,,,
f1,60,60,60,60,60,60,60,60,60,60,60,60
f2,60,60,60,60,60,60,60,60,60,60,60,60
f3,60,60,60,60,60,60,60,60,60,60,60,60
f4,60,60,60,60,60,60,60,60,60,60,60,60
f5,60,60,60,60,60,60,60,60,60,60,60,60
f6,60,60,60,60,60,60,60,60,60,60,60,60
f7,60,60,60,60,60,60,60,60,60,60,60,60
f8,60,60,60,60,60,60,60,60,60,60,60,60
f9,60,60,60,60,60,60,60,60,60,60,60,60


In [12]:
f_all['r1_rubric_city_param'] = f_all['r1_city'] + '. ' + f_all['r1_rubric'] + '. ' + f_all['r1_param'] 
f_all['r2_rubric_city_param'] = f_all['r2_city'] + '. ' + f_all['r2_rubric'] + '. ' + f_all['r2_param'] 
f_cr = f_all[['r1_city','r1_rubric','r1_param']].drop_duplicates()
cr = list(zip(f_cr['r1_city'].values,f_cr['r1_rubric'].values,f_cr['r1_param'].values))

def replace_slash(s):
    return s[::-1].replace('/', 'и', 1)[::-1].replace('СПБ', 'Санкт-Петербург')


docx_name = 'Формулы_корреляции_Москва_230526_1100.docx'
doc = docx.Document('files/report.docx')
doc.save(docx_name)
p = doc.add_paragraph()
for city, rubric, param in cr:
    f_i = f_all[(f_all.r1_city == city) & (f_all.r1_rubric == rubric)]
    cond = f_i['group'].iloc[0] in ('f1', 'f2', 'f3')
    cond2 = f_i['group'].iloc[0] in ('f7', 'f8', 'f9')
    if cond:
        col1 = 'r1_rubric'
        col2 = 'r2_rubric'
    elif cond2:
        col1 = 'r1_rubric_city_param'
        col2 = 'r2_rubric_city_param'
    else:
        col1 = 'r1_rubric_city'
        col2 = 'r2_rubric_city'
        
    head = f_i['group'].iloc[0] +': '+ f_i[col1].iloc[0] + ' = '
    left_sq = 'f{'
    body = '\n\t,'.join([f'{i:.4f} x {replace_slash(j)}' for i, j in list(zip(f_i['corr'].values, f_i[col2].values))])
    right_sq = '}\n'
    runner = p.add_run(head)
    runner.font.highlight_color = WD_COLOR_INDEX.YELLOW
    runner2 = p.add_run(left_sq)
    runner2.italic = True
    p.add_run(body)
    runner3 = p.add_run(right_sq)
    runner3.italic = True
    doc.save(docx_name)

In [143]:
f_cr = f_all[['r1_city','r1_rubric']].drop_duplicates()
cr = list(zip(f_cr['r1_city'].values, f_cr['r1_rubric'].values))

def replace_slash(s):
    return s[::-1].replace('/', 'и', 1)[::-1].replace('СПБ', 'Санкт-Петербург')


docx_name = 'Формулы_корреляции_Москва_230526_1100.docx'
doc = docx.Document('files/report.docx')
doc.save(docx_name)
p = doc.add_paragraph()
for city, rubric in cr:
    f_i = f_all[(f_all.r1_city == city) & (f_all.r1_rubric == rubric)]
    cond = f_i['group'].iloc[0] not in ('f4', 'f5', 'f6')
    col1 = 'r1_rubric' if cond else 'r1_rubric_city'
    col2 = 'r2_rubric' if cond else 'r2_rubric_city'
    head = f_i['group'].iloc[0] +': '+ f_i[col1].iloc[0] + ' = '
    left_sq = 'f{'
    body = '\n\t,'.join([f'{i:.4f} x {replace_slash(j)}' for i, j in list(zip(f_i['corr'].values, f_i[col2].values))])
    right_sq = '}\n'
    runner = p.add_run(head)
    runner.font.highlight_color = WD_COLOR_INDEX.YELLOW
    runner2 = p.add_run(left_sq)
    runner2.italic = True
    p.add_run(body)
    runner3 = p.add_run(right_sq)
    runner3.italic = True
    doc.save(docx_name)

1.	Направленность:
f1 •	прямая – прогноз активности оцениваемого бизнеса строится на наиболее коррелирующих показателях без учёта их природы
f2 •	обратная – предсказание основывается на зависимостях обратной природы
f3 •	смешанная – формула сочетает наибольшие пары положительных и отрицательных корреляций.
2.	Однородность:
f7•	органическая – для построения тренда используются смысловые ряды переменных: запросы базируются на запросах, пользователи на пользователях, динамика на динамике, схожесть на схожести
f8•	синтетическая – в формулу замешиваются показатели наибольшей предсказательной силы, без учёта их физической природы
f9•	адаптационная – задействуется оптимизационный алгоритм вариативности, чередующий переменные, не допуская преобладания единственного типа.
3.	Комбинаторность:
•	низкая – для предсказания востребованности одного бизнеса в городе используются другие бизнесы этого же населённого пункта
•	средняя – тренд деятельности в городе строится на основе других активностей в иных населённых пунктах
f6•	максимальная – прогноз строится на любом количестве бизнесов любых населённых пунктов для получения максимальной сходимости.
